<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch10/section_90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 90. 次単語予測
“The movie was full of”に続くトークン（トークン列ではなく一つのトークンであることに注意せよ）として適切なもの上位10個と、その確率（尤度）を求めよ。ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

In [31]:
!pip install -U transformers

In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [33]:
model_id = 'gpt2'

In [34]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id)
model.eval()

In [36]:
text = "The movie was full of"

encodings = tokenizer(text, return_tensors='pt')

In [37]:
input_ids = encodings['input_ids'].squeeze(0)
tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(f"tokens: {tokens}")

tokens: ['The', 'Ġmovie', 'Ġwas', 'Ġfull', 'Ġof']


In [38]:
with torch.no_grad():
  outputs = model(**encodings) # **はkeyword引数を展開するpythonの記法

`outputs.logits[0, -1]`とは何か？
- logitsのshapeは(batch_size, seq_len, vocab_size)となっている
- `[0, -1]`は省略形で、実際は`[0, -1, :]`と同じである
- よってbatch_sizeの0番目、seq_lenの-1番目、vocab_sizeは全てとなる
  - batch_sizeの0番目: 今回は1つしかないため変わらない
  - seq_lenの-1番目: 最後のトークン
なぜ最後のトークンなのか
- GPTは各位置tで次の単語の予測をしている
- 今回は文章に続く単語を予測しないといけないので文章の中で最後の文字の次の単語の予測を取得しないといけない

In [39]:
logits = outputs.logits[0, -1]
print(f"last logits: {logits}")
print(f"All logits: {outputs.logits}")


last logits: tensor([-102.3519, -101.2763, -106.8403,  ..., -103.5513, -107.0933,
        -102.1604])
All logits: tensor([[[ -36.2874,  -35.0114,  -38.0793,  ...,  -40.5163,  -41.3760,
           -34.9193],
         [ -91.6833,  -91.4114,  -94.6155,  ..., -100.2464,  -97.9104,
           -92.4564],
         [-125.3571, -124.7983, -127.3332,  ..., -134.9384, -128.8435,
          -126.3980],
         [ -52.1309,  -54.5084,  -62.0278,  ...,  -62.6903,  -61.0079,
           -54.3363],
         [-102.3519, -101.2763, -106.8403,  ..., -103.5513, -107.0933,
          -102.1604]]])


語彙サイズの尤度（確率）が手に入るので、各インデックスがそのままconvert_ids_to_tokensに渡せる

In [45]:
prods = torch.softmax(logits, dim=-1) # logits.shape = (vocab_size,)
print(f"prods: {prods}")

prods: tensor([2.5071e-06, 7.3499e-06, 2.8175e-08,  ..., 7.5550e-07, 2.1876e-08,
        3.0362e-06])


topkは上位k個の要素と、その要素のindexを返す

In [41]:
topk = torch.topk(prods, k=10)
print(f"topk: {topk}")

topk: torch.return_types.topk(
values=tensor([0.0219, 0.0186, 0.0115, 0.0109, 0.0107, 0.0105, 0.0100, 0.0074, 0.0074,
        0.0067]),
indices=tensor([14532,  1049, 22051,  2089, 24072, 10288,  1257, 14733,   366,   262]))


:>12 とは？
- Python の f-string の書式指定
  - : 書式指定を始める
  - > 右寄せ
  - 12 幅 12 文字

!r とは？
- repr() を使って表示する指定
- 先頭の空白や改行など、人間が通常見落とす文字も確認できる

.6f とは？
- 小数点以下 6 桁まで表示（四捨五入）

tokenizer.decode はなぜ tokenizer.decode([idx.item()]) とするのか？
- tokenizer.decode は「トークン列（list[int]）」から文字列に変換する関数
- 1トークンでも list に包む必要がある
- idx は Tensor なので、まず item() で Python の int に変換する

item() とは？
- torch.Tensor（1要素）を Python のスカラー型（int / float）に変換するメソッド


In [47]:
print("\nTop-10 next tokens:")
for p, idx in zip(topk.values, topk.indices):
  token_str = tokenizer.convert_ids_to_tokens(idx.item()) # item()でpython互換に変換する
  decoded = tokenizer.decode([idx.item()])
  print(f"{token_str:>12} | {decoded!r:>12} | {p.item():.6f}")


Top-10 next tokens:
      Ġjokes |     ' jokes' | 0.021892
      Ġgreat |     ' great' | 0.018644
     Ġlaughs |    ' laughs' | 0.011524
        Ġbad |       ' bad' | 0.010874
  Ġsurprises | ' surprises' | 0.010667
 Ġreferences | ' references' | 0.010528
        Ġfun |       ' fun' | 0.009992
      Ġhumor |     ' humor' | 0.007415
          Ġ" |         ' "' | 0.007408
        Ġthe |       ' the' | 0.006709
